# Cleaning Flights Data

In [0]:
## Path configuration

BRONZE_PATH = "dbfs:/FileStore/vijay_project/bronze/streaming_data/flights"
SILVER_PATH = "dbfs:/FileStore/vijay_project/silver/flights"

In [0]:
## Imports

 
from pyspark.sql import DataFrame
from pyspark.sql.functions import (
    col, lit, concat_ws, to_date, when, expr
)
from pyspark.sql.types import IntegerType
from pyspark.sql.window import Window
 

In [0]:
## Reading from bronze and loading into data frame

bronze_df = spark.read.format("delta").load(BRONZE_PATH)
silver_df = bronze_df.withColumn(
    "FLIGHT_DATE",
    to_date(concat_ws("-", col("YEAR"), col("MONTH"), col("DAY")), "yyyy-M-d")
)

In [0]:
## function to time stamp creation(yyyy-mm-dd HHmm)

def hhmm_to_timestamp_safe(df: DataFrame, time_col: str) -> DataFrame:
    return df.withColumn(
        f"{time_col}_TS",
        expr(f"""
            CASE
                WHEN {time_col} IS NULL THEN NULL
                WHEN trim({time_col}) = '' THEN NULL
                WHEN length(lpad({time_col}, 4, '0')) != 4 THEN NULL
                ELSE try_to_timestamp(
                    concat(FLIGHT_DATE, ' ', lpad({time_col}, 4, '0')),
                    'yyyy-MM-dd HHmm'
                )
            END
        """)
    )
 
TIME_COLUMNS = [
    "SCHEDULED_DEPARTURE",
    "DEPARTURE_TIME",
    "WHEELS_OFF",
    "WHEELS_ON",
    "SCHEDULED_ARRIVAL",
    "ARRIVAL_TIME"
]
 
for c in TIME_COLUMNS:
    if c in silver_df.columns:
        silver_df = hhmm_to_timestamp_safe(silver_df, c)

# Drop raw HHmm columns
silver_df = silver_df.drop(*[c for c in TIME_COLUMNS if c in silver_df.columns])

In [0]:
## Type Casting ( converting string to int)

DURATION_COLUMNS = [
    "DEPARTURE_DELAY",
    "TAXI_OUT",
    "TAXI_IN",
    "AIR_TIME",
    "ELAPSED_TIME",
    "ARRIVAL_DELAY",
    "AIR_SYSTEM_DELAY",
    "SECURITY_DELAY",
    "AIRLINE_DELAY",
    "LATE_AIRCRAFT_DELAY",
    "WEATHER_DELAY"
]
 
existing_columns = set(silver_df.columns)
 
# Cast existing columns
for c in DURATION_COLUMNS:
    if c in existing_columns:
        silver_df = silver_df.withColumn(c, col(c).cast(IntegerType()))
 
# Add missing duration columns as NULL (stable schema)
existing_columns = set(silver_df.columns)
for c in DURATION_COLUMNS:
    if c not in existing_columns:
        silver_df = silver_df.withColumn(c, lit(None).cast(IntegerType()))
 

In [0]:
## Normalizing negative delays

silver_df = (
    silver_df
    .withColumn(
        "DEPARTURE_DELAY",
        when(col("DEPARTURE_DELAY").isNull(), None)
        .when(col("DEPARTURE_DELAY") < 0, 0)
        .otherwise(col("DEPARTURE_DELAY"))
    )
    .withColumn(
        "ARRIVAL_DELAY",
        when(col("ARRIVAL_DELAY").isNull(), None)
        .when(col("ARRIVAL_DELAY") < 0, 0)
        .otherwise(col("ARRIVAL_DELAY"))
    )
)

In [0]:
## cleaning the data( removing nulls)

silver_df = silver_df.filter(
    col("FLIGHT_DATE").isNotNull() &
    col("AIRLINE").isNotNull() &
    col("FLIGHT_NUMBER").isNotNull() &
    col("ORIGIN_AIRPORT").isNotNull() &
    col("DESTINATION_AIRPORT").isNotNull()
)

In [0]:
## deduplicating the data

DEDUP_KEYS = [
    "FLIGHT_DATE",
    "AIRLINE",
    "FLIGHT_NUMBER",
    "ORIGIN_AIRPORT",
    "DESTINATION_AIRPORT"
]
 
silver_df = (
    silver_df
    .withColumn(
        "rn",
        expr("""
            row_number() OVER (
                PARTITION BY FLIGHT_DATE, AIRLINE, FLIGHT_NUMBER,
                             ORIGIN_AIRPORT, DESTINATION_AIRPORT
                ORDER BY INGESTED_AT DESC
            )
        """)
    )
    .filter(col("rn") == 1)
    .drop("rn")
)



In [0]:
## defining final columns


FINAL_COLUMNS = [
    "FLIGHT_DATE",
    "YEAR", "MONTH", "DAY", "DAY_OF_WEEK",
    "AIRLINE", "FLIGHT_NUMBER", "TAIL_NUMBER",
    "ORIGIN_AIRPORT", "DESTINATION_AIRPORT",
 
    "SCHEDULED_DEPARTURE_TS",
    "DEPARTURE_TIME_TS",
    "WHEELS_OFF_TS",
    "WHEELS_ON_TS",
    "SCHEDULED_ARRIVAL_TS",
    "ARRIVAL_TIME_TS",
 
    "DEPARTURE_DELAY", "ARRIVAL_DELAY",
    "TAXI_OUT", "TAXI_IN", "AIR_TIME", "ELAPSED_TIME",
 
    "CANCELLED", "DIVERTED", "CANCELLATION_REASON",
 
    "INGESTED_AT", "SOURCE_FILE"
]
 
existing_columns = set(silver_df.columns)
final_columns_safe = [c for c in FINAL_COLUMNS if c in existing_columns]
 
silver_df = silver_df.select(*final_columns_safe)
 

In [0]:
## writing into silver layer

(
    silver_df.write
    .format("delta")
    .mode("overwrite")
    .partitionBy("YEAR", "MONTH")
    .option("overwriteSchema", "true")
    .save(SILVER_PATH)
)

# Cleaning Airlines and Airports Data

In [0]:
# imports

from pyspark.sql.functions import col, trim, upper, current_timestamp, row_number
from pyspark.sql.window import Window

In [0]:
# Defining Paths
bronze_airlines_path = "dbfs:/FileStore/vijay_project/bronze/batch_data/airlines"
bronze_airports_path = "dbfs:/FileStore/vijay_project/bronze/batch_data/airports"

silver_airlines_path = "dbfs:/FileStore/vijay_project/silver/airlines"
silver_airports_path = "dbfs:/FileStore/vijay_project/silver/airports"

### Airlines

In [0]:
# read and load data

airlines_df = spark.read.format("delta").load(bronze_airlines_path)

# --- Clean Columns ---
airlines_df = (
    airlines_df
    .withColumn("IATA_CODE", upper(trim(col("IATA_CODE"))))
    .withColumn("AIRLINE", trim(col("AIRLINE")))
)

# --- Deduplicate ---
w_airl = Window.partitionBy("IATA_CODE").orderBy(col("LOAD_TIMESTAMP").desc())

airlines_silver = (
    airlines_df
    .withColumn("rn", row_number().over(w_airl))
    .filter(col("rn") == 1)
    .drop("rn")
)

# --- Add Silver Metadata ---
airlines_silver = airlines_silver.withColumn("INGESTED_AT", current_timestamp())

# --- Write Silver Table ---
(
    airlines_silver.write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .save(silver_airlines_path)
)

### Airports

In [0]:
# read and load data

airports_df = spark.read.format("delta").load(bronze_airports_path)

# --- Clean Columns ---
airports_df = (
    airports_df
    .withColumn("IATA_CODE", upper(trim(col("IATA_CODE"))))
    .withColumn("AIRPORT", trim(col("AIRPORT")))
    .withColumn("CITY", trim(col("CITY")))
    .withColumn("STATE", trim(col("STATE")))
    .withColumn("COUNTRY", trim(col("COUNTRY")))
    .withColumn("LATITUDE", trim(col("LATITUDE")))
    .withColumn("LONGITUDE", trim(col("LONGITUDE")))
)

# --- Deduplicate ---
w_airp = Window.partitionBy("IATA_CODE").orderBy(col("LOAD_TIMESTAMP").desc())

airports_silver = (
    airports_df
    .withColumn("rn", row_number().over(w_airp))
    .filter(col("rn") == 1)
    .drop("rn")
)

# --- Add Silver Metadata ---
airports_silver = airports_silver.withColumn("INGESTED_AT", current_timestamp())

# --- Write Silver Table ---
(
    airports_silver.write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .save(silver_airports_path)
)
